In [ ]:
import pandas as pd
import json
import numpy  as np

import copy
from sklearn.metrics                  import classification_report
from sklearn.feature_extraction.text  import TfidfVectorizer
import torch


import pandas as pd
import numpy as np


from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
training_data_path = '/content/drive/MyDrive/NLP/rumoureval2019/rumoureval-2019-training-data'
test_data_path     = '/content/drive/MyDrive/NLP/rumoureval2019/rumoureval-2019-test-data'

twitter_trainingDev_data_path = training_data_path + '/twitter-english'
twitter_test_data_path        = test_data_path + '/twitter-en-test-data'

path_train_key = '/content/drive/MyDrive/NLP/rumoureval2019/rumoureval-2019-training-data/train-key.json'
path_dev_key   = '/content/drive/MyDrive/NLP/rumoureval2019/rumoureval-2019-training-data/dev-key.json'
path_test_key  = '/content/drive/MyDrive/NLP/rumoureval2019/final-eval-key.json'

reddit_train_data_path  =  training_data_path + '/reddit-training-data'
reddit_dev_data_path    =  training_data_path + '/reddit-dev-data'
reddit_test_data_path   =  test_data_path     + '/reddit-test-data'

In [ ]:
train_key_df = pd.read_json(path_train_key)
dev_key_df = pd.read_json(path_dev_key)
test_key_df = pd.read_json(path_test_key)

In [ ]:
train_key_df['subtaskbenglish'].value_counts()

true          144
unverified    104
false          79
Name: subtaskbenglish, dtype: int64

In [ ]:
def processTwitterKeyDataFrame(key_df, datasetType):
    key_taskb_df = pd.DataFrame(key_df['subtaskbenglish'].dropna())

    key_taskb_df = key_taskb_df.reset_index()
    key_taskb_df = key_taskb_df.rename(columns={'index': 'id', 'subtaskbenglish': 'label'})
    
    if datasetType == 'train':
        twitter_key_tasks_df = key_taskb_df[0:297] 
    elif datasetType == 'dev':
        twitter_key_tasks_df = key_taskb_df[0:28] 
    elif datasetType == 'test':
        twitter_key_tasks_df =  key_taskb_df[0:56] 
    return twitter_key_tasks_df

In [ ]:
key_taskb_df = pd.DataFrame(test_key_df['subtaskbenglish'].dropna())

In [ ]:
key_taskb_df[0:56]

,subtaskbenglish
443938194715713536,true
774165935041093633,true
784071228248109057,true
784118929799073793,true
784216706080178176,true
784316107645603840,true
828933929080086528,false
830469481621241857,false
852844529778601985,false
872300483272114177,true


In [ ]:
twitter_train_key_df = processTwitterKeyDataFrame(train_key_df, 'train')
twitter_dev_key_df = processTwitterKeyDataFrame(dev_key_df, 'dev')
twitter_test_key_df = processTwitterKeyDataFrame(test_key_df, 'test')

In [ ]:
twitter_train_key_df

,id,label
0,500319801344929795,unverified
1,553589051044151296,true
2,524925730053181440,unverified
3,525060425184858112,true
4,544382892378714113,true
...,...,...
292,580371845997682688,true
293,580339825649291264,false
294,580882341880446977,true
295,580339547269144576,false


In [ ]:
def processTwitterSourcePosts(twitter_dataset_path):
    twitter_dirs = next(os.walk(twitter_dataset_path))[1]
    twitter_dirs_sorted = sorted(twitter_dirs)
    
    twitter_src_dirs = []
    twitter_src_posts = []
    
    for directory in twitter_dirs_sorted:
        tweet_src_path = twitter_dataset_path + '/' + directory + '/source-tweet' 
        twitter_src_dirs.extend(next(os.walk(tweet_src_path))[2])
  
    src_tweet_files_sorted = sorted(twitter_src_dirs)
    
    for file in src_tweet_files_sorted:
        paths = twitter_dataset_path + '/' + file.split('.')[0] + '/source-tweet' + '/' + file
        tweet_post_dict = {}
        
       
        with open(paths) as f:
            for line in f:
                src = json.loads(line)
                text = src['text']
                inre = src['in_reply_to_status_id']
                tid = src['id']
                
                tweet_post_dict['text'] = text 
                tweet_post_dict['id'] = tid   
                tweet_post_dict['inre'] = inre
                twitter_src_posts.append(tweet_post_dict)
    print(twitter_src_posts) 
    print(len(twitter_src_posts))
    return twitter_dirs_sorted, twitter_src_posts

In [ ]:
twitter_issue_dirs = next(os.walk(twitter_trainingDev_data_path))[1]
twitter_issue_dirs = sorted(twitter_issue_dirs)
source_posts = pd.DataFrame()
for dir in twitter_issue_dirs:
  print("* current dir is * "+ dir)
  twitter_trainDev_dirs_sorted , twitter_trainDev_src_posts = processTwitterSourcePosts(twitter_trainingDev_data_path+"/"+dir)
  source_posts = source_posts.append(twitter_trainDev_src_posts)
  source_posts = source_posts.reset_index(drop=True)

* current dir is * charliehebdo
[{'text': 'France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58', 'id': 552783667052167168, 'inre': None}, {'text': 'BREAKING: 10 reportedly shot dead at Paris HQ of French weekly Charlie Hebdo http://t.co/5F1DOwzoCQ', 'id': 552785375161499649, 'inre': None}, {'text': 'Appalled by the attack on Charlie Hebdo in Paris, 10 - probably journalists - now confirmed dead. An attack on free speech everywhere.', 'id': 552788945017516032, 'inre': None}, {'text': "BREAKING: At least 10 killed in shooting at French satirical newspaper Charlie Hebdo, Paris prosecutor's office says. http://t.co/mnAeA7j7fY", 'id': 552791196247269378, 'inre': None}, {'text': 'Eleven dead in shooting at Paris offices of satirical magazine Charlie Hebdo – live updates http://t.co/gOtSqbwROc', 'id': 552791578893619200, 'inre': None}, {'text': 'BREAKING Charlie Hebdo latest: 11 dead 10 wounded (five critical) 

In [ ]:
twitter_trainDev_src_posts_df = source_posts


In [ ]:
twitter_trainDev_src_posts_df

,text,id,inre
0,France: 10 people dead after shooting at HQ of...,552783667052167168,None
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,552785375161499649,None
2,Appalled by the attack on Charlie Hebdo in Par...,552788945017516032,None
3,BREAKING: At least 10 killed in shooting at Fr...,552791196247269378,None
4,Eleven dead in shooting at Paris offices of sa...,552791578893619200,None
...,...,...,...
319,"Haron Monis,Hostage-taker in #Sydney: Fled #Ir...",544515538383564801,None
320,#BREAKING: Police have confirmed that the #Syd...,544517264054423552,None
321,#BREAKING: At least two people have died follo...,544518335019229184,None
322,"#sydneysiege is over. 2 confirmed dead, #PrayF...",544520042810200064,None


In [ ]:
twitter_issue_test_dirs = next(os.walk(twitter_test_data_path))[1]
twitter_issue_test_dirs = sorted(twitter_issue_test_dirs)
print(twitter_issue_test_dirs)
source_posts_test = pd.DataFrame()
for dir in twitter_issue_test_dirs:
  print("* current dir is * "+ dir)
  twitter_test_dirs_sorted , twitter_test_src_posts = processTwitterSourcePosts(twitter_test_data_path+"/"+dir)
  source_posts_test = source_posts_test.append(twitter_test_src_posts)
  source_posts_test = source_posts_test.reset_index(drop=True)

['afghanistan', 'african-american', 'ark-park', 'barbara-bush', 'blm-hurricane', 'cape-tornadoes', 'clinton-ads', 'clinton-florida', 'cnn-lies', 'fire-ants', 'fukushima-dump', 'fukushima-falling', 'harvey-donations', 'immigrants-wildfires', 'inflatable-dam', 'irma-cat-6', 'lawn-tornado', 'lightning-river', 'mississippi-dry', 'nat-geo-footage', 'obama-harvey', 'osteen-yacht', 'pence-sacrifice', 'red-cross', 'shark-hurricane', 'texas-mosque', 'virgin-weapons', 'wildfires-deduction']
* current dir is * afghanistan
[{'text': '#Breaking: Pentagon releases video of the “mother of all bombs” being dropped in Afghanistan https://t.co/GaXwhpWDmb', 'id': 852844529778601985, 'inre': None}]
1
* current dir is * african-american
[{'text': 'Rep. Sheila Jackson Lee has no shame. I still laugh over her insistence years ago that hurricanes have African American sounding names!', 'id': 443938194715713536, 'inre': None}, {'text': 'ICYMI: "Rep Sheila Jackson Lee (D-Tx) Wants Hurricanes To Be Given \'Afric

In [ ]:
twitter_test_src_posts_df = source_posts_test

In [ ]:
twitter_test_src_posts_df

,text,id,inre
0,#Breaking: Pentagon releases video of the “mot...,852844529778601985,None
1,Rep. Sheila Jackson Lee has no shame. I still ...,443938194715713536,None
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",774165935041093633,None
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",909084965035487232,None
4,Noah’s Ark Theme Park Destroyed in a Flood? ht...,897502435593265152,None
5,@burnodo I also haven't forgotten her quote ab...,986750534886424576,986749210136600576.0
6,"@carridea @Darph_Nader ""And so many of the peo...",986801053168488449,986800951406268416.0
7,My lasting impression of Barbara Bush was the ...,987323105524768769,NaN
8,"A remembrance.\n\n""What I'm hearing, which is ...",987801763850809344,NaN
9,Black Lives Matter THUGS Blocking Emergency Cr...,901939374375370752,None


In [ ]:
def twitterCleanDf(src_posts_df):
    twitter_data = src_posts_df


    twitter_data['id']   = twitter_data.id.astype(str)
    twitter_data['inre'] = twitter_data.inre.astype(str)
  
    twitter_clean_data = pd.DataFrame(twitter_data)
   
    twitter_clean_data.id   = twitter_clean_data.id.str.strip()     
    twitter_clean_data.inre = twitter_clean_data.inre.str.strip()   
    return twitter_clean_data

In [ ]:
twitter_clean_trainDev_df = twitterCleanDf(twitter_trainDev_src_posts_df)
twitter_clean_test_df     = twitterCleanDf(twitter_test_src_posts_df)


twitter_train_withKeys_df = pd.merge(twitter_clean_trainDev_df, twitter_train_key_df, how = 'inner', on = "id", )
twitter_test_withKeys_df  = pd.merge(twitter_clean_test_df, twitter_test_key_df, how = 'inner', on = "id", )

In [ ]:
twitter_train_withKeys_df['label'].value_counts()

true          136
unverified     98
false          62
Name: label, dtype: int64

In [ ]:
twitter_test_withKeys_df

,text,id,inre,label
0,#Breaking: Pentagon releases video of the “mot...,852844529778601985,None,false
1,Rep. Sheila Jackson Lee has no shame. I still ...,443938194715713536,None,true
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",774165935041093633,None,true
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",909084965035487232,None,true
4,Noah’s Ark Theme Park Destroyed in a Flood? ht...,897502435593265152,None,false
5,@burnodo I also haven't forgotten her quote ab...,986750534886424576,9.867492101366006e+17,true
6,"@carridea @Darph_Nader ""And so many of the peo...",986801053168488449,9.868009514062684e+17,true
7,My lasting impression of Barbara Bush was the ...,987323105524768769,nan,true
8,"A remembrance.\n\n""What I'm hearing, which is ...",987801763850809344,nan,true
9,Black Lives Matter THUGS Blocking Emergency Cr...,901939374375370752,None,false


In [ ]:
twitter_train_withKeys_df

,text,id,inre,label
0,France: 10 people dead after shooting at HQ of...,552783667052167168,None,true
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,552785375161499649,None,true
2,BREAKING: At least 10 killed in shooting at Fr...,552791196247269378,None,true
3,Eleven dead in shooting at Paris offices of sa...,552791578893619200,None,true
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,552792544132997121,None,true
...,...,...,...,...
291,"Haron Monis,Hostage-taker in #Sydney: Fled #Ir...",544515538383564801,None,true
292,#BREAKING: Police have confirmed that the #Syd...,544517264054423552,None,true
293,#BREAKING: At least two people have died follo...,544518335019229184,None,true
294,"#sydneysiege is over. 2 confirmed dead, #PrayF...",544520042810200064,None,true


In [ ]:
def processRedditKeyDataFrame(key_df, datasetType):
    key_taskb_df = pd.DataFrame(key_df['subtaskbenglish'].dropna())
     
    key_taskb_df = key_taskb_df.reset_index()
    key_taskb_df = key_taskb_df.rename(columns={'index': 'id', 'subtaskbenglish': 'label'})
    
    if datasetType   ==  'train':
        reddit_key_tasks_df = key_taskb_df[297:] 
    elif datasetType ==  'dev':
        reddit_key_tasks_df = key_taskb_df[28:] 
    elif datasetType ==  'test':
        reddit_key_tasks_df =  key_taskb_df[56:] 
    return reddit_key_tasks_df

In [ ]:
train_key_df['subtaskbenglish'][297:]

544270541725511680      NaN
524944416721862656      NaN
500288271650684928      NaN
500323321519800321      NaN
500478509366915072      NaN
                      ...  
8gp0d7                 true
8i274x                false
8n9173                false
8unvgg                false
8yktu5                false
Name: subtaskbenglish, Length: 4920, dtype: object

In [ ]:
train_key_df['subtaskbenglish'].value_counts()

true          144
unverified    104
false          79
Name: subtaskbenglish, dtype: int64

In [ ]:
reddit_train_key_df = processRedditKeyDataFrame(train_key_df, 'train')
reddit_dev_key_df   = processRedditKeyDataFrame(dev_key_df, 'dev')
reddit_test_key_df  = processRedditKeyDataFrame(test_key_df, 'test')

In [ ]:
reddit_train_key_df = reddit_train_key_df.reset_index(drop=True)
reddit_dev_key_df = reddit_dev_key_df.reset_index(drop=True)
reddit_test_key_df = reddit_test_key_df.reset_index(drop=True)

In [ ]:
reddit_train_key_df['label'].value_counts()

false         17
true           7
unverified     6
Name: label, dtype: int64

In [ ]:
def processRedditSourcePosts(reddit_dataset_path):
    reddit_dirs = next(os.walk(reddit_dataset_path))[1] 
    reddit_dirs_sorted = sorted(reddit_dirs)
    
    reddit_src_dirs  = []
    reddit_src_posts = []
    
    for directory in reddit_dirs_sorted:
        reddit_src_path = reddit_dataset_path + '/' + directory + '/source-tweet' 
        reddit_src_dirs.append(next(os.walk(reddit_src_path))[2])
    
    src_reddit_files = []
    for sdirs in reddit_src_dirs:
        for i in sdirs:
            src_reddit_files.append(i)
    src_reddit_files_sorted = sorted(src_reddit_files)
    
    for file in src_reddit_files_sorted:
        paths = reddit_dataset_path + '/' + file.split('.')[0] + '/source-tweet' + '/' + file 
        reddit_post_dict = {}
        
        
        with open(paths) as f:
            for line in f:
                src = json.loads(line)
                text = src['data']['children'][0]['data']['title']
                rid = src['data']['children'][0]['data']['id']
                
                reddit_post_dict['text'] = text  
                reddit_post_dict['id'] = rid    
                reddit_post_dict['inre'] = 'None'
                reddit_src_posts.append(reddit_post_dict)
      
    return reddit_dirs_sorted, reddit_src_posts

In [ ]:
reddit_train_dirs_sorted , reddit_train_src_posts = processRedditSourcePosts(reddit_train_data_path)
reddit_train_src_posts_df = pd.DataFrame(reddit_train_src_posts)

reddit_dev_dirs_sorted , reddit_dev_src_posts = processRedditSourcePosts(reddit_dev_data_path)
reddit_dev_src_posts_df = pd.DataFrame(reddit_dev_src_posts)

reddit_test_dirs_sorted , reddit_test_src_posts = processRedditSourcePosts(reddit_test_data_path)
reddit_test_src_posts_df = pd.DataFrame(reddit_test_src_posts)

In [ ]:
reddit_train_src_posts_df

,text,id,inre
0,Even ants won't eat aspartame!,18dmb4,None
1,"""Cancer is a fungus"" - this idea from the 60s ...",1hzz6y,None
2,repost from TIL : 'financial guru' Robert Kiyo...,1i8cy7,None
3,Is it true that if you are not a member of the...,1i8ljs,None
4,How much truth is there in the statement that ...,22o24j,None
5,Debunk This: Microwaves are bad because microw...,249p6c,None
6,Debunk this: Nicotine isn't really bad for you...,25bvmb,None
7,Would Labour win if young people voted?,46uw4y,None
8,California To Allow Illegal Immigrants To Vote...,46yxoy,None
9,'Queen backs Brexit' - The Sun front page tomo...,49l01s,None


In [ ]:
def redditCleanDf(src_posts_df):
    reddit_data = src_posts_df


    reddit_data['id'] = reddit_data.id.astype(str)
    reddit_data['inre'] = reddit_data.inre.astype(str)
    
    reddit_clean_data = pd.DataFrame(reddit_data)
    
    reddit_clean_data.id = reddit_clean_data.id.str.strip()    
    reddit_clean_data.inre = reddit_clean_data.inre.str.strip() 
    return reddit_clean_data

In [ ]:
reddit_clean_train_df = redditCleanDf(reddit_train_src_posts_df)
reddit_clean_dev_df   = redditCleanDf(reddit_dev_src_posts_df)
reddit_clean_test_df  = redditCleanDf(reddit_test_src_posts_df)

reddit_train_withKeys_df = pd.merge(reddit_clean_train_df, reddit_train_key_df, how = 'inner', on = "id", )
reddit_dev_withKeys_df   = pd.merge(reddit_clean_dev_df, reddit_dev_key_df, how = 'inner', on = "id", )
reddit_test_withKeys_df  = pd.merge(reddit_clean_test_df, reddit_test_key_df, how = 'inner', on = "id", )

In [ ]:
reddit_train_withKeys_df

,text,id,inre,label
0,Even ants won't eat aspartame!,18dmb4,None,false
1,"""Cancer is a fungus"" - this idea from the 60s ...",1hzz6y,None,false
2,repost from TIL : 'financial guru' Robert Kiyo...,1i8cy7,None,true
3,Is it true that if you are not a member of the...,1i8ljs,None,unverified
4,How much truth is there in the statement that ...,22o24j,None,unverified
5,Debunk This: Microwaves are bad because microw...,249p6c,None,false
6,Debunk this: Nicotine isn't really bad for you...,25bvmb,None,false
7,Would Labour win if young people voted?,46uw4y,None,unverified
8,California To Allow Illegal Immigrants To Vote...,46yxoy,None,false
9,'Queen backs Brexit' - The Sun front page tomo...,49l01s,None,false


In [ ]:
def label_to_int(label):
  if label   == 'true':
    return 0
  elif label == 'false':
    return 1
  elif label == 'unverified':
    return 2
  

def processText(text):
  text = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", "$URL$",text.strip())
  text = re.sub(r"(@[A-Za-z0-9]+)", "$MENTION$", text.strip())

  return text

In [ ]:
def processStanceData(twitterDf, RedditDf):
  frames = [twitterDf, RedditDf]

  resultDf = pd.concat(frames)                                                      
  result1  = resultDf.replace(np.nan, '', regex=True)                               
  result1 = result1.reset_index(drop=True)
  result1['labelvalue'] = result1.label.apply(label_to_int)                      
  data = result1[['text','id','label','labelvalue']].copy()

  data['text'] = data.text.apply(processText)
  return data

In [ ]:
trainDf = processStanceData(twitter_train_withKeys_df,reddit_train_withKeys_df)
trainDf

,text,id,label,labelvalue
0,France: 10 people dead after shooting at HQ of...,552783667052167168,true,0
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,552785375161499649,true,0
2,BREAKING: At least 10 killed in shooting at Fr...,552791196247269378,true,0
3,Eleven dead in shooting at Paris offices of sa...,552791578893619200,true,0
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,552792544132997121,true,0
...,...,...,...,...
321,[Serious] Is it true that 85% of people can on...,8gp0d7,true,0
322,Did Eric Schneiderman Help NXIVM Sell Child Se...,8i274x,false,1
323,Debunk this: The direction you sleep matters,8n9173,false,1
324,Is it true police officers must hit a quota at...,8unvgg,false,1


In [ ]:
trainDf.to_csv('/content/drive/MyDrive/NLP/csvfiles/taskb/trainingData.csv', encoding='utf-8', index=False)

In [ ]:
trainDf['label'].value_counts()

true          143
unverified    104
false          79
Name: label, dtype: int64

In [ ]:
testDf = processStanceData(twitter_test_withKeys_df,reddit_test_withKeys_df)
testDf

,text,id,label,labelvalue
0,#Breaking: Pentagon releases video of the “mot...,852844529778601985,false,1
1,Rep. Sheila Jackson Lee has no shame. I still ...,443938194715713536,true,0
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",774165935041093633,true,0
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",909084965035487232,true,0
4,Noah’s Ark Theme Park Destroyed in a Flood? $URL$,897502435593265152,false,1
...,...,...,...,...
76,People who can't smell. Is it true that there ...,8m9y0o,true,0
77,Great Pyramid of Giza can focus electromagneti...,938o5h,true,0
78,Is it true that the federal government require...,ayn70,false,1
79,Is it true Romney has Caymen bank accounts?,wfajw,true,0


In [ ]:
testDf.to_csv('/content/drive/MyDrive/NLP/csvfiles/taskb/testData.csv', encoding='utf-8', index=False)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/taskb/trainingData.csv')

In [ ]:
train

,text,id,label,labelvalue
0,France: 10 people dead after shooting at HQ of...,552783667052167168,true,0
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,552785375161499649,true,0
2,BREAKING: At least 10 killed in shooting at Fr...,552791196247269378,true,0
3,Eleven dead in shooting at Paris offices of sa...,552791578893619200,true,0
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,552792544132997121,true,0
...,...,...,...,...
321,[Serious] Is it true that 85% of people can on...,8gp0d7,true,0
322,Did Eric Schneiderman Help NXIVM Sell Child Se...,8i274x,false,1
323,Debunk this: The direction you sleep matters,8n9173,false,1
324,Is it true police officers must hit a quota at...,8unvgg,false,1


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/taskb/testData.csv')

In [ ]:
test

,text,id,label,labelvalue
0,#Breaking: Pentagon releases video of the “mot...,852844529778601985,false,1
1,Rep. Sheila Jackson Lee has no shame. I still ...,443938194715713536,true,0
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",774165935041093633,true,0
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",909084965035487232,true,0
4,Noah’s Ark Theme Park Destroyed in a Flood? $URL$,897502435593265152,false,1
...,...,...,...,...
76,People who can't smell. Is it true that there ...,8m9y0o,true,0
77,Great Pyramid of Giza can focus electromagneti...,938o5h,true,0
78,Is it true that the federal government require...,ayn70,false,1
79,Is it true Romney has Caymen bank accounts?,wfajw,true,0


In [ ]:
train['label'].isnull().sum()

0

In [ ]:
x_train = train['text'].tolist()
y = train['labelvalue'].fillna(0)
y_train = y.tolist()



x_test = test['text'].tolist()
y_test = test['labelvalue'].tolist()

tfidf         = TfidfVectorizer(min_df = 10, max_df = 0.5, ngram_range=(1,2))
x_train_feats = tfidf.fit(x_train)

print(x_train_feats)
print(len(x_train_feats.get_feature_names_out()))

x_train_transform = x_train_feats.transform(x_train)

tfidf_transform_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(x_train_transform)).float()
print(x_train_transform.shape)


x_test = tfidf.transform(x_test)


x_test = torch.tensor(scipy.sparse.csr_matrix.todense(x_test)).float()

TfidfVectorizer(max_df=0.5, min_df=10, ngram_range=(1, 2))
102
(326, 102)


In [ ]:

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train_transform,y_train)

predictions_SVM = SVM.predict(x_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  41.9753086419753


In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)
SVM.fit(x_train_transform,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(x_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  41.9753086419753


In [ ]:
class_probabilities = SVM.predict_proba(x_test)

In [ ]:
predictions_SVM

array([0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2])

In [ ]:
class_probabilities

array([[0.52855302, 0.09245462, 0.37899236],
       [0.46685654, 0.30519835, 0.22794511],
       [0.42295983, 0.21115118, 0.36588898],
       [0.42063093, 0.12052115, 0.45884792],
       [0.3885192 , 0.21025406, 0.40122674],
       [0.25492315, 0.3344264 , 0.41065045],
       [0.32668543, 0.27382531, 0.39948926],
       [0.31694369, 0.14902207, 0.53403424],
       [0.46727399, 0.24328312, 0.28944288],
       [0.3613709 , 0.12490136, 0.51372773],
       [0.3613709 , 0.12490136, 0.51372773],
       [0.67150134, 0.0624591 , 0.26603956],
       [0.32094351, 0.58668494, 0.09237155],
       [0.12795589, 0.35173051, 0.5203136 ],
       [0.30322968, 0.45475187, 0.24201845],
       [0.3741752 , 0.18158821, 0.44423658],
       [0.33914676, 0.29175529, 0.36909795],
       [0.35091225, 0.2681113 , 0.38097644],
       [0.35091225, 0.2681113 , 0.38097644],
       [0.35091225, 0.2681113 , 0.38097644],
       [0.28449587, 0.29057229, 0.42493184],
       [0.40754032, 0.2807799 , 0.31167978],
       [0.

In [ ]:
class_probabilities[1][predictions_SVM[1]]

0.46084108406461416

In [ ]:
predictions_SVM

array([0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2])

In [ ]:
cscore = []
for i in range(len(predictions_SVM)):
  if predictions_SVM[i] == 2:
    cscore.append(0)
  else:
    cscore.append(class_probabilities[i][predictions_SVM[i]])

In [ ]:
cscore[5]

0

In [ ]:
oneList = [1]*len(cscore)

In [ ]:
oneList

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(oneList,cscore, squared=False)


In [ ]:
rms

0.6243523195029409

In [ ]:
cr = classification_report(y_test, predictions_SVM)
print(cr)

              precision    recall  f1-score   support

           0       0.44      0.52      0.48        31
           1       0.57      0.42      0.49        40
           2       0.07      0.10      0.08        10

    accuracy                           0.42        81
   macro avg       0.36      0.35      0.35        81
weighted avg       0.46      0.42      0.43        81



In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(x_train_transform,y_train)
preds = clf.predict(x_test)
print("mlp Accuracy Score -> ",accuracy_score(preds, y_test)*100)

mlp Accuracy Score ->  43.20987654320987


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
cr = classification_report(y_test, preds)
print(cr)

              precision    recall  f1-score   support

           0       0.42      0.26      0.32        31
           1       0.62      0.62      0.62        40
           2       0.09      0.20      0.13        10

    accuracy                           0.43        81
   macro avg       0.38      0.36      0.36        81
weighted avg       0.48      0.43      0.45        81



In [ ]:
preds

array([0, 0, 2, 2, 1, 2, 0, 2, 0, 2, 2, 0, 1, 2, 1, 2, 1, 0, 0, 0, 2, 1,
       0, 1, 1, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 2, 1, 1, 2])

In [ ]:
len(preds)

81

In [ ]:
c=0
p = list(preds)
for i in range(len(preds)):
  if preds[i] == y_test[i]:
    c +=1
    print(preds[i],y_test[i])
print(c)

0 0
1 1
0 0
0 0
1 1
2 2
0 0
1 1
0 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 1
1 1
0 0
1 1
1 1
1 1
2 2
0 0
1 1
35


In [ ]:
y_test

[1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 1]

In [ ]:
class_probabilities = clf.predict_proba(x_test)

In [ ]:
class_probabilities

array([[9.34323828e-01, 1.10418157e-04, 6.55657539e-02],
       [8.66090697e-01, 1.31241095e-01, 2.66820801e-03],
       [6.68133262e-02, 3.27274476e-02, 9.00459226e-01],
       [2.73136407e-01, 3.03483432e-02, 6.96515250e-01],
       [2.79738818e-01, 3.62691635e-01, 3.57569546e-01],
       [1.92912564e-01, 2.19902877e-01, 5.87184559e-01],
       [6.20666466e-01, 9.95699744e-02, 2.79763560e-01],
       [3.24741017e-02, 8.62065960e-04, 9.66663832e-01],
       [7.52054765e-01, 2.08112785e-01, 3.98324501e-02],
       [1.56026433e-01, 3.06458428e-03, 8.40908983e-01],
       [1.56026433e-01, 3.06458428e-03, 8.40908983e-01],
       [7.57639885e-01, 4.76968763e-04, 2.41883146e-01],
       [2.95772838e-03, 9.96960905e-01, 8.13665233e-05],
       [6.25872661e-04, 2.12890118e-01, 7.86484009e-01],
       [1.51051880e-02, 9.80444459e-01, 4.45035252e-03],
       [1.16806391e-01, 3.37153434e-01, 5.46040175e-01],
       [5.70329649e-02, 7.89697304e-01, 1.53269731e-01],
       [4.89525956e-01, 4.03107

In [ ]:
cscoreMLP = []
for i in range(len(preds)):
  if preds[i] == 2:
    cscoreMLP.append(0)
  else:
    cscoreMLP.append(class_probabilities[i][preds[i]])

In [ ]:
cscoreMLP

[0.9343238279646502,
 0.8660906965308149,
 0,
 0,
 0.3626916353173756,
 0,
 0.6206664658349468,
 0,
 0.752054764625087,
 0,
 0,
 0.7576398854361921,
 0.9969609050957294,
 0,
 0.9804444594470706,
 0,
 0.7896973042482119,
 0.48952595606931554,
 0.48952595606931554,
 0.48952595606931554,
 0,
 0.5682853942186564,
 0.3869781009676343,
 0.5744613043973454,
 0.5744613043973454,
 0,
 0,
 0,
 0.9936054036775154,
 0.9620545397318523,
 0.9620545397318523,
 0.7421773937989014,
 0,
 0,
 0.9974761764695358,
 0.9769789560275509,
 0.9993751388790669,
 0.9968461053546048,
 0.9998022641796498,
 0.9998022641796498,
 0.42895104653560157,
 0.7207082808116302,
 0.8743056074158722,
 0.9821555675436862,
 0,
 0.9997105822981628,
 0.9997105822981628,
 0.9988374575757588,
 0.9993696155108155,
 0.6842605354640628,
 0,
 0.5744613043973454,
 0,
 0,
 0.9890190980582088,
 0.7234572912875762,
 0.5810099952915009,
 0.9963026920085197,
 0.43512545792435,
 0.958819599899386,
 0.7209312857888913,
 0.8302849692622418,
 0.9

In [ ]:
oneList = [1]*len(cscoreMLP)

In [ ]:
from sklearn.metrics import mean_squared_error

rmsMLP = mean_squared_error(oneList,cscoreMLP, squared=False)


In [ ]:
rmsMLP

0.5712503086670723